# Preparing Data

In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 
import seaborn as sns
import matplotlib.pyplot as plt

import wandb
from kaggle_secrets import UserSecretsClient
import collections
from collections import OrderedDict

In [2]:
# wandb setting
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("wandb")
os.environ["WANDB_API_KEY"] = secret_value

wandb.login()

wandb: Currently logged in as: soobin. Use `wandb login --relogin` to force relogin


True

In [3]:
#device 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
#HyperParameter 설정
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'PATIENCE': 10,
    'LEARNING_RATE':4e-4,
    'BATCH_SIZE': 128,
    'SEED':2023
}

In [5]:
#Seed 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

# Train Data

## Extract data with sum of labels 3 or less

In [7]:
train_df1 = pd.read_csv('/dacon-blocks/train.csv')

train_df1['cnt']=train_df1.sum(axis=1)

train_df1 = train_df1[train_df1['cnt'] < 4]
train_df1 = train_df1.drop(columns='cnt')
train_df1

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,./train/TRAIN_00000.jpg,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,./train/TRAIN_00001.jpg,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,./train/TRAIN_00002.jpg,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,./train/TRAIN_00003.jpg,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,./train/TRAIN_00004.jpg,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27429,TRAIN_27429,./train/TRAIN_27429.jpg,0,0,0,0,0,0,0,1,1,1
27430,TRAIN_27430,./train/TRAIN_27430.jpg,0,0,0,0,0,0,0,1,1,1
27431,TRAIN_27431,./train/TRAIN_27431.jpg,0,0,0,0,0,0,0,1,1,1
27432,TRAIN_27432,./train/TRAIN_27432.jpg,0,0,0,0,0,0,0,1,1,1


## Extract data with total

In [8]:
train_df2 = pd.read_csv('/dacon-blocks/train.csv')
train_df2

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,./train/TRAIN_00000.jpg,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,./train/TRAIN_00001.jpg,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,./train/TRAIN_00002.jpg,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,./train/TRAIN_00003.jpg,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,./train/TRAIN_00004.jpg,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
32989,TRAIN_32989,./train/TRAIN_32989.jpg,1,0,0,0,1,0,1,1,1,1
32990,TRAIN_32990,./train/TRAIN_32990.jpg,1,0,0,0,0,1,1,1,1,1
32991,TRAIN_32991,./train/TRAIN_32991.jpg,1,0,0,0,0,1,1,1,1,1
32992,TRAIN_32992,./train/TRAIN_32992.jpg,1,0,0,0,0,1,1,1,1,1


## Data load

In [9]:
train_316 = train_df1.copy()
train_own = train_df1.copy()
train_new = train_df1.copy()
train_clean = train_df1.copy()
train_new_white = train_df2.copy()
train_own_white = train_df2.copy()

print(train_316.shape, train_own.shape, train_new.shape, train_clean.shape, train_new_white.shape, train_own_white.shape)

(9434, 12) (9434, 12) (9434, 12) (32994, 12) (32994, 12) (9434, 12)


In [10]:
train_316['img_path'] = train_316['img_path'].apply(lambda x: '/dacon-blocks/train_316/' + x[8:])
train_316['img_path']

0        /kaggle/input/dacon-blocks/train_316/TRAIN_000...
1        /kaggle/input/dacon-blocks/train_316/TRAIN_000...
2        /kaggle/input/dacon-blocks/train_316/TRAIN_000...
3        /kaggle/input/dacon-blocks/train_316/TRAIN_000...
4        /kaggle/input/dacon-blocks/train_316/TRAIN_000...
                               ...                        
27429    /kaggle/input/dacon-blocks/train_316/TRAIN_274...
27430    /kaggle/input/dacon-blocks/train_316/TRAIN_274...
27431    /kaggle/input/dacon-blocks/train_316/TRAIN_274...
27432    /kaggle/input/dacon-blocks/train_316/TRAIN_274...
27433    /kaggle/input/dacon-blocks/train_316/TRAIN_274...
Name: img_path, Length: 9434, dtype: object

In [11]:
train_own['img_path'] = train_own['img_path'].apply(lambda x: '/dacon-blocks/train_own/' + x[8:])
train_own['img_path']

0        /kaggle/input/dacon-blocks/train_own/TRAIN_000...
1        /kaggle/input/dacon-blocks/train_own/TRAIN_000...
2        /kaggle/input/dacon-blocks/train_own/TRAIN_000...
3        /kaggle/input/dacon-blocks/train_own/TRAIN_000...
4        /kaggle/input/dacon-blocks/train_own/TRAIN_000...
                               ...                        
27429    /kaggle/input/dacon-blocks/train_own/TRAIN_274...
27430    /kaggle/input/dacon-blocks/train_own/TRAIN_274...
27431    /kaggle/input/dacon-blocks/train_own/TRAIN_274...
27432    /kaggle/input/dacon-blocks/train_own/TRAIN_274...
27433    /kaggle/input/dacon-blocks/train_own/TRAIN_274...
Name: img_path, Length: 9434, dtype: object

In [12]:
train_new['img_path'] = train_new['img_path'].apply(lambda x: '/dacon-blocks/train_new/' + x[8:])
train_new['img_path']

0        /kaggle/input/dacon-blocks/train_new/TRAIN_000...
1        /kaggle/input/dacon-blocks/train_new/TRAIN_000...
2        /kaggle/input/dacon-blocks/train_new/TRAIN_000...
3        /kaggle/input/dacon-blocks/train_new/TRAIN_000...
4        /kaggle/input/dacon-blocks/train_new/TRAIN_000...
                               ...                        
27429    /kaggle/input/dacon-blocks/train_new/TRAIN_274...
27430    /kaggle/input/dacon-blocks/train_new/TRAIN_274...
27431    /kaggle/input/dacon-blocks/train_new/TRAIN_274...
27432    /kaggle/input/dacon-blocks/train_new/TRAIN_274...
27433    /kaggle/input/dacon-blocks/train_new/TRAIN_274...
Name: img_path, Length: 9434, dtype: object

In [15]:
train_clean['img_path'] = train_clean['img_path'].apply(lambda x: '/dacon-blocks/train_clean/train_clean' + x[7:])
train_clean['img_path']

0        /kaggle/input/dacon-blocks/train_clean/train_c...
1        /kaggle/input/dacon-blocks/train_clean/train_c...
2        /kaggle/input/dacon-blocks/train_clean/train_c...
3        /kaggle/input/dacon-blocks/train_clean/train_c...
4        /kaggle/input/dacon-blocks/train_clean/train_c...
                               ...                        
27429    /kaggle/input/dacon-blocks/train_clean/train_c...
27430    /kaggle/input/dacon-blocks/train_clean/train_c...
27431    /kaggle/input/dacon-blocks/train_clean/train_c...
27432    /kaggle/input/dacon-blocks/train_clean/train_c...
27433    /kaggle/input/dacon-blocks/train_clean/train_c...
Name: img_path, Length: 9434, dtype: object

In [13]:
train_new_white['img_path'] = train_new_white['img_path'].apply(lambda x: '/dacon-blocks/whitegrid_train/train_new_white' + x[7:])
train_new_white['img_path']

0        /kaggle/input/dacon-blocks/whitegrid_train/tra...
1        /kaggle/input/dacon-blocks/whitegrid_train/tra...
2        /kaggle/input/dacon-blocks/whitegrid_train/tra...
3        /kaggle/input/dacon-blocks/whitegrid_train/tra...
4        /kaggle/input/dacon-blocks/whitegrid_train/tra...
                               ...                        
32989    /kaggle/input/dacon-blocks/whitegrid_train/tra...
32990    /kaggle/input/dacon-blocks/whitegrid_train/tra...
32991    /kaggle/input/dacon-blocks/whitegrid_train/tra...
32992    /kaggle/input/dacon-blocks/whitegrid_train/tra...
32993    /kaggle/input/dacon-blocks/whitegrid_train/tra...
Name: img_path, Length: 32994, dtype: object

In [14]:
train_own_white['img_path'] = train_own_white['img_path'].apply(lambda x: '/dacon-blocks/whitegrid_train/train_own_white' + x[7:])
train_own_white['img_path']

0        /kaggle/input/dacon-blocks/whitegrid_train/tra...
1        /kaggle/input/dacon-blocks/whitegrid_train/tra...
2        /kaggle/input/dacon-blocks/whitegrid_train/tra...
3        /kaggle/input/dacon-blocks/whitegrid_train/tra...
4        /kaggle/input/dacon-blocks/whitegrid_train/tra...
                               ...                        
32989    /kaggle/input/dacon-blocks/whitegrid_train/tra...
32990    /kaggle/input/dacon-blocks/whitegrid_train/tra...
32991    /kaggle/input/dacon-blocks/whitegrid_train/tra...
32992    /kaggle/input/dacon-blocks/whitegrid_train/tra...
32993    /kaggle/input/dacon-blocks/whitegrid_train/tra...
Name: img_path, Length: 32994, dtype: object

## Merging train data

In [16]:
train = pd.concat([train_own, train_316, train_new, train_clean, train_new_white, train_own_white])
train.shape

(103724, 12)

In [17]:
train

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,/kaggle/input/dacon-blocks/train_own/TRAIN_000...,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,/kaggle/input/dacon-blocks/train_own/TRAIN_000...,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,/kaggle/input/dacon-blocks/train_own/TRAIN_000...,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,/kaggle/input/dacon-blocks/train_own/TRAIN_000...,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,/kaggle/input/dacon-blocks/train_own/TRAIN_000...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27429,TRAIN_27429,/kaggle/input/dacon-blocks/train_clean/train_c...,0,0,0,0,0,0,0,1,1,1
27430,TRAIN_27430,/kaggle/input/dacon-blocks/train_clean/train_c...,0,0,0,0,0,0,0,1,1,1
27431,TRAIN_27431,/kaggle/input/dacon-blocks/train_clean/train_c...,0,0,0,0,0,0,0,1,1,1
27432,TRAIN_27432,/kaggle/input/dacon-blocks/train_clean/train_c...,0,0,0,0,0,0,0,1,1,1


# Validation data

In [ ]:
train_raw = pd.read_csv('/dacon-blocks/train.csv')

print(train_raw.shape)

## Data Load

In [18]:
train_raw['img_path'] = train_raw['img_path'].apply(lambda x: '/kaggle/input/dacon-blocks/train/' + x[8:])
train_raw.head()

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,/kaggle/input/dacon-blocks/train/TRAIN_00000.jpg,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,/kaggle/input/dacon-blocks/train/TRAIN_00001.jpg,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,/kaggle/input/dacon-blocks/train/TRAIN_00002.jpg,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,/kaggle/input/dacon-blocks/train/TRAIN_00003.jpg,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,/kaggle/input/dacon-blocks/train/TRAIN_00004.jpg,1,0,0,0,0,0,0,0,0,0


In [19]:
#10개의 라벨을 하나의 열에 setting
cols =['A','B','C','D','E','F','G','H','I','J']
train_raw['labels'] = train_raw[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
train_raw['labels']

#label을 기준으로 val data를 일정한 비율로 추출하기 위해 설정
labels = train_raw['labels'].to_list()
c = collections.Counter(labels)

val = pd.DataFrame()
keys = collections.Counter(labels).keys()

#0.2의 비율로 val data 추출
for i in keys:
    val = val.append(train_raw[train_raw['labels'].values == i].sample(frac = 0.2))

#private data
private = train_raw.drop(val.index)
private = private.reset_index(drop=True)
private.to_csv('model_private.csv',index=False)

print(train.shape, val.shape, private.shape)

(103724, 12)
(6856, 13)
(26138, 13)


In [20]:
# 데이터 추출 및 합성으로 인해 index 초기화
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

In [21]:
#학습에 필요한 라벨 설정
def get_labels(df):
    return df.iloc[:,2:-1].values

train_labels = train.iloc[:,2:].values
val_labels = get_labels(val)

# model

## Data Custom (csv 파일과 실제 이미지 연결)

In [22]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

## Image Augmentation

In [23]:
train_transform = A.Compose([A.CenterCrop(350,350,p=1),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.ColorJitter(brightness=0.2, contrast=0.2, hue=0.2),
                            A.HorizontalFlip(p=0.5),
                            A.RandomBrightnessContrast((-0.05,0.05), p=0.5),
                            ToTensorV2() 
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

## DataLoader

In [24]:
train_dataset = CustomDataset(train['img_path'].values, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=10)

val_dataset = CustomDataset(val['img_path'].values, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=10)

## Add model layer

In [25]:
fc = nn.Sequential(OrderedDict([
('bn1', nn.BatchNorm1d(1000)),
('drop1', nn.Dropout(p=0.5)),
('linear1', nn.Linear(1000, 256)),
('drop2', nn.Dropout(p=0.5)),
('linear2', nn.Linear(256, 64)),
('drop3', nn.Dropout(p=0.5)),
('linear3', nn.Linear(64, 10))
]))

# resnet34.fc = fc

## BaseModel (resnext50_32*4d)

In [26]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=10):
        super(BaseModel, self).__init__()
        self.backbone = models.resnext50_32x4d(pretrained=True)
        for param in self.backbone.parameters():
            param.requiresGrad = False
        
        self.fc = fc
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(x)
        return x

## EarlyStopping

In [27]:
class EarlyStopping:
    def __init__(self, patience=CFG['PATIENCE'], verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

## train & validation

In [28]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCEWithLogitsLoss().to(device)
    
    best_val_acc = 0
    best_model = None
    early_stopping = EarlyStopping(patience=CFG['PATIENCE'], verbose = True)
    
    wandb.init(project='dacon-blocks', entity='soobin',name='resnext50_labels3_total',
      config={"learning_rate": CFG['LEARNING_RATE'],
              "epochs": CFG['EPOCHS'],
              "batch_size": CFG['BATCH_SIZE']
      })
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        log_dict = {}
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        #wandb 기록
        log_dict['epoch'] = epoch
        log_dict['train loss'] = np.mean(train_loss)
        log_dict['validation loss'] = _val_loss
        log_dict['validation acc'] = _val_acc
        wandb.log(log_dict)
        
        early_stopping(_val_loss, model)
        
        # epoch 5마다 모델 save
        if epoch % 5 == 0:
             torch.save(model.state_dict(), '/kaggle/working/checkpoint_ep_%s.pt' %epoch)
                
        if early_stopping.early_stop:
            print("Early stopping")
            break

        if scheduler is not None:
            scheduler.step(_val_acc)
            
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model
    
    return best_model

In [29]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            probs = model(imgs)
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs >= 0.5
            batch_acc = (labels == preds).mean()
            
            val_acc.append(batch_acc)
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)
    
    return _val_loss, _val_acc

# Train Model

In [30]:
#모델 학습
model = BaseModel()
model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

  0%|          | 0/811 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.22839] Val Loss : [0.15590] Val ACC : [0.93933]


...

  0%|          | 0/54 [00:00<?, ?it/s]

Epoch [22], Train Loss : [0.00839] Val Loss : [0.06119] Val ACC : [0.99100]
EarlyStopping counter: 9 out of 10


  0%|          | 0/811 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

Epoch [23], Train Loss : [0.00850] Val Loss : [0.06618] Val ACC : [0.99080]
EarlyStopping counter: 10 out of 10
Early stopping


In [31]:
torch.save(model.state_dict(),'resnext50_model.pth')

# Prediction model

In [32]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions

## Prediction private data

In [34]:
#private 계산
test_dataset = CustomDataset(private['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = 256, shuffle=False, num_workers=0)

preds = inference(model, test_loader, device)

prediction = private.iloc[:,:-1]
prediction.iloc[:,2:] = preds
# submit.to_csv('model_prediction.csv', index=False)

# 실제값과 예측값 비교
cols =['A','B','C','D','E','F','G','H','I','J']
prediction['labels'] = prediction[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

print('예측값: ', prediction['labels'].head())
print('실제값: ', private['labels'].head())

a=[]
for idx in range(len(private)):
    if private['labels'][idx]!= prediction['labels'][idx]:
        a.append(idx)
print('private 틀린 개수: ', len(a))
print('private 점수: ', (len(private)-len(a))/len(private))

  0%|          | 0/103 [00:00<?, ?it/s]

예측값:  0    1 0 0 0 0 0 0 0 0 0
1    1 0 0 0 0 0 0 0 0 0
2    1 0 0 0 0 0 0 0 0 0
3    1 0 0 0 0 0 0 0 0 0
4    1 0 0 0 0 0 0 0 0 0
Name: labels, dtype: object
실제값:  0    1 0 0 0 0 0 0 0 0 0
1    1 0 0 0 0 0 0 0 0 0
2    1 0 0 0 0 0 0 0 0 0
3    1 0 0 0 0 0 0 0 0 0
4    1 0 0 0 0 0 0 0 0 0
Name: labels, dtype: object
private 틀린 개수:  1879
private 점수:  0.928112326880404


## Prediction test data

In [ ]:
# test 예측
test = pd.read_csv('/dacon-blocks/test.csv')
test['img_path'] = test['img_path'].apply(lambda x: '/dacon-blocks/test_whitegrid'+ x[6:])
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=5)

preds = inference(model, test_loader, device)

submit = pd.read_csv('/dacon-blocks/sample_submission.csv')
submit.iloc[:,1:] = preds
submit.head()
submit.to_csv('test_prediction.csv', index=False)